In [39]:
import pandas as pd
import os
import numpy as np
from itertools import product


In [40]:
def replace_char(s, position, new_char):
    return s[:position] + new_char + s[position + 1:]

original_text = "abcdefg"
modified_text = replace_char(original_text, 1, "Z")
print(modified_text)  # Output: "aZcdefg"

aZcdefg


In [42]:
ALL_AAS = ("A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y")
q = 20

df_protein_meta = pd.read_csv("ArnoldDatasetsMeta.csv")
df_protein_meta.head()
for index, row in df_protein_meta.iterrows():
    protein_name = row['proteinName']
    n = int(row['numPositions'])

    file_path = f'TG_results/{protein_name}_q{q}_n{n}.npy'
    dataset_file_path = f'ArnoldDatasets/{protein_name}.csv'

    all_combos_list = list(product(ALL_AAS, repeat=n))

    all_combos_str = []
    for i in all_combos_list:
        all_combos_str.append(''.join(i))
    all_combos_str = np.array(all_combos_str)

    df = pd.read_csv(dataset_file_path)
    print("df with stop codons", len(df))
    print(df.head())
    print()
    df = df[df['AAs'].isin(all_combos_str)]
    fitness_scores = df['fitness'].to_numpy()
    assert len(df) == len(fitness_scores)
    print("df with no stop codons", len(df))
    print(df.head())
    print()

    df_all = pd.DataFrame({'AAs':all_combos_str})
    df_merged = df.merge(df_all, on='AAs', how='outer')
    print("df with all combos but before imputation", len(df_merged))
    print(df_merged.head())
    print()

    df_nans = df_merged[df_merged['fitness'].isnull()]
    assert (len(df_nans)+len(fitness_scores)) == q**n

    for combo in df_nans['AAs']:
        mut_combos = np.empty((1, q*n), dtype=str)
        for pos in range(n):
            for aa in ALL_AAS:
                mut_combo = replace_char(combo, pos, aa)
                mut_combos = np.append(mut_combos, mut_combo)
        mut_fitness_vals = df[df['AAs'].isin(mut_combos)]['fitness']
        if len(mut_fitness_vals) == 0:
            df_merged.loc[df_merged['AAs'] == combo, 'fitness'] = 0
        else:
            df_merged.loc[df_merged['AAs'] == combo, 'fitness'] = mut_fitness_vals.mean()

    print("df after imputation", len(df_merged))
    print(df_merged.head())
    print()

    np.save(file_path, df_merged['fitness'].to_numpy())
    print(protein_name, q, n)

df with stop codons 9261
   AAs AA1 AA2 AA3   fitness  active            muts  n_mut
0  ***   *   *   *  0.126525   False  A26*:D27*:L28*      3
1  **A   *   *   A  0.123933   False  A26*:D27*:L28A      3
2  **C   *   *   C  0.123492   False  A26*:D27*:L28C      3
3  **D   *   *   D  0.125148   False  A26*:D27*:L28D      3
4  **E   *   *   E  0.124846   False  A26*:D27*:L28E      3

df with no stop codons 8000
     AAs AA1 AA2 AA3   fitness  active       muts  n_mut
463  AAA   A   A   A  0.128071   False  D27A:L28A      2
464  AAC   A   A   C  0.126441   False  D27A:L28C      2
465  AAD   A   A   D  0.126975   False  D27A:L28D      2
466  AAE   A   A   E  0.125828   False  D27A:L28E      2
467  AAF   A   A   F  0.125645   False  D27A:L28F      2

df with all combos but before imputation 8000
   AAs AA1 AA2 AA3   fitness  active       muts  n_mut
0  AAA   A   A   A  0.128071   False  D27A:L28A      2
1  AAC   A   A   C  0.126441   False  D27A:L28C      2
2  AAD   A   A   D  0.126975   F